# Azure Responses Client Direct Usage Example

**Copyright (c) Microsoft. All rights reserved.**

Demonstrates direct `AzureOpenAIResponsesClient` usage for structured response generation with Azure OpenAI models.
Shows function calling capabilities with custom business logic.

## Features Demonstrated
- Direct Azure OpenAI Responses API usage
- Function calling with custom tools
- Structured output using Pydantic models
- Streaming and non-streaming responses
- Azure CLI credential authentication

## Setup and Imports

Import required libraries for Azure OpenAI integration, function calling, and structured outputs.

In [ ]:
from random import randint
from typing import Annotated

from agent_framework import ChatResponse
from agent_framework.azure import AzureOpenAIResponsesClient
from azure.identity import AzureCliCredential
from pydantic import BaseModel, Field


## Define Custom Tools

Create a function that will be called by the model to get weather information.
The function uses type annotations for automatic schema generation.

In [2]:
def get_weather(
    location: Annotated[str, Field(description="The location to get the weather for.")],
) -> str:
    """Get the weather for a given location."""
    conditions = ["sunny", "cloudy", "rainy", "stormy"]
    temp = randint(10, 30)
    return f"The weather in {location} is {conditions[randint(0, 3)]} with a high of {temp}°C."

## Define Output Structure

Create a Pydantic model to enforce structured output format.

In [3]:
class OutputStruct(BaseModel):
    """Structured output for weather information."""

    location: str
    weather: str

## Initialize Azure OpenAI Client

Create the client using Azure CLI credentials. Make sure you've run `az login` before executing this cell.

**Note:** You can replace `AzureCliCredential` with other authentication options like `DefaultAzureCredential`, `ManagedIdentityCredential`, etc.

In [4]:
# Initialize the Azure OpenAI Responses Client
client = AzureOpenAIResponsesClient(credential=AzureCliCredential())

## Example 1: Streaming Response

Get a streaming response with function calling and structured output.

In [5]:
async def streaming_example():
    message = "What's the weather in Amsterdam and in Paris?"
    print(f"User: {message}\n")

    response = await ChatResponse.from_chat_response_generator(
        client.get_streaming_response(message, tools=get_weather, response_format=OutputStruct),
        output_format_type=OutputStruct,
    )

    print(f"Assistant: {response.value}")
    return response


# Run the streaming example
streaming_response = await streaming_example()

User: What's the weather in Amsterdam and in Paris?

Assistant: location='Amsterdam, Paris' weather='Amsterdam: stormy, high 28°C. Paris: rainy, high 30°C.'


## Example 2: Non-Streaming Response

Get a non-streaming response with the same capabilities.

In [ ]:
async def non_streaming_example():
    message = "What's the weather in Amsterdam and in Paris?"
    print(f"User: {message}\n")

    response = await client.get_response(message, tools=get_weather, response_format=OutputStruct)

    print(f"Assistant: {response.value}")
    return response


# Run the non-streaming example
non_streaming_response = await non_streaming_example()

## Inspect Response Structure

Examine the structured output returned by the model.

In [ ]:
# Inspect the response value (Pydantic model)
print("Response type:", type(non_streaming_response.value))
print("Response data:", non_streaming_response.value.model_dump_json(indent=2))

## Complete Example Function

A complete example combining all the concepts above.

In [ ]:
async def main(stream: bool = True) -> None:
    """
    Main function demonstrating Azure OpenAI Responses Client usage.

    Args:
        stream: Whether to use streaming responses (default: True)
    """
    # Initialize client
    client = AzureOpenAIResponsesClient(credential=AzureCliCredential())

    message = "What's the weather in Amsterdam and in Paris?"
    print(f"User: {message}\n")

    if stream:
        print("[Using streaming response]\n")
        response = await ChatResponse.from_chat_response_generator(
            client.get_streaming_response(message, tools=get_weather, response_format=OutputStruct),
            output_format_type=OutputStruct,
        )
    else:
        print("[Using non-streaming response]\n")
        response = await client.get_response(
            message, tools=get_weather, response_format=OutputStruct
        )

    print(f"Assistant: {response.value}")
    return response


# Run with streaming
result = await main(stream=True)

## Try Different Queries

Experiment with different weather queries.

In [ ]:
async def query_weather(location: str):
    """Query weather for a specific location."""
    client = AzureOpenAIResponsesClient(credential=AzureCliCredential())
    message = f"What's the weather in {location}?"
    print(f"User: {message}\n")

    response = await client.get_response(message, tools=get_weather, response_format=OutputStruct)
    print(f"Assistant: {response.value}\n")
    return response


# Try different locations
await query_weather("London")
await query_weather("Tokyo")
await query_weather("New York")

## Summary

This notebook demonstrated:

1. **Direct Azure OpenAI Responses Client usage** - No agent framework required for simple scenarios
2. **Function calling** - Custom tools with automatic schema generation from type hints
3. **Structured outputs** - Pydantic models for reliable response parsing
4. **Streaming and non-streaming** - Both modes supported with same API
5. **Azure authentication** - Using Azure CLI credentials (extensible to other methods)

### Next Steps

- Explore more complex function calling scenarios
- Add multiple tools and let the model decide which to use
- Integrate with the full AgenticFleet orchestration system
- Add error handling and retry logic for production use